In [169]:
#3SLA_Theory_based_computation
import math

In [256]:
##Defining the SLAs
##Common part
retrievalPrice = 75 ##Price which CMP pays for the retrieval of a data item from the source
gapTime = 0.15 ##This valiable is defining the caching strategy; 0 means full coverage, inf means reactive
##gapTime is measured after the end of freshness period of SLA1


# freshPeriodListOrig = [0.1, 0.35, 0.48, 0.55, 0.67]
# lambdaList = [8,12,18,25,40]
# requestPriceList = [180,130,80, 40,20]
# penaltyList = [190, 150, 100, 50, 30]

freshPeriodList = [1.148119540723993, 1.2353714971882412, 2.415473087993526, 2.5124435788965718, 2.722023056477838, 3.7868621337274377, 5.393024357881993, 6.135265515050152, 6.159169396795528, 9.243347449403675]
lambdaList = [0.151420272526263, 0.21572343300636931, 0.3646613248132008, 0.4928951904713921, 0.5187212232549007, 0.6440280930797282, 0.6934648944550585, 0.7304405364772717, 0.8101433346335052, 0.9453486581780832]
requestPriceList = [10, 17, 26, 51, 60, 61, 67, 74, 80, 91]
penaltyList = [11, 23, 42, 55, 59, 61, 79, 79, 87, 99]


planningPeriod = 600

##Length of Refresh period (RP) 
freshPeriodListOrig = freshPeriodList.copy()
RP = freshPeriodList[0] + gapTime
maxSegments = 0


In [252]:
##Counting maximum possible number of segments
def countMaxSegments():
    global maxSegments 
    maxSegments = 0
    print('RP: ',RP)
    for i in range(len(freshPeriodList)):
        maxSegments = i + 1
        #print(maxSegments)
        if freshPeriodList[i] >= RP:
            print(freshPeriodList[i])
            break
    if freshPeriodList[len(freshPeriodList) - 1] < RP:
        maxSegments = maxSegments + 1
    print("maxSegments:", maxSegments)
    return maxSegments


In [225]:
print(countMaxSegments())

RP:  1.13302352744846
1.4015184673113645
maxSegments: 1
1


In [226]:
##We need to re-calculate the freshness periods in case when the automatic refresh is cutting one of the segments
def periodsCut():
    global freshPeriodList
    for i in range(0, len(freshPeriodList)):
        if freshPeriodList[i] > RP:
            freshPeriodList[i] = RP  

In [227]:
periodsCut()

In [228]:
freshPeriodListOrig

[1.4015184673113645,
 1.7010399730562764,
 2.61569923498713,
 3.754060057588381,
 3.8337316761972806,
 4.4621240670124855,
 5.283225827043081,
 6.682348333559199,
 8.035658718691975,
 9.954928639654922]

In [229]:
freshPeriodList

[1.13302352744846,
 1.13302352744846,
 1.13302352744846,
 1.13302352744846,
 1.13302352744846,
 1.13302352744846,
 1.13302352744846,
 1.13302352744846,
 1.13302352744846,
 1.13302352744846]

In [230]:
print(maxSegments)

1


In [231]:
#P_Miss_sk_with_Miss_k = lambdaS/lambda1+...+LambdaK  [Eq. D]
## It is a probability that SLA s produced a miss in segment K, in case that a miss really happened in segment K

In [232]:
##Correct, verified
def P_Miss_sk_with_Miss_k(s, k):  
    if k <= s:
        return 0
    lambdaS = lambdaList[s-1]      
    sumLamb1ToLambK = 0
    #if k > len(lambdaList):
    #    k=len(lambdaList)
    for i in range(k-1):
        sumLamb1ToLambK += lambdaList[i]
        #print("lambda",i+1)            
    result = lambdaS / sumLamb1ToLambK 
    return result


In [233]:
##Now let's define a function for finding P[Misss_m..l(u,v)]   Eq. F
#it is the arrival probability of all sla's (m to l) from time u to time v

In [234]:
##Verified, correct
def P_Miss_ml_uv(m,l,u,v):
    
    ##add a check that time of SLA_l expiry is less than u
    if freshPeriodList[l-1] > u or freshPeriodList[m-1] > u :
        return 'freshPeriod of SLA m or SLA l should be less than time u'
    if u > v:
        return ('P_Miss_ml_uv:  u should be smaller than v')
        
    sum_over_exp = 0
    lambda_k = 0
    k = m
    while k <= l: ## k < L (not one)
        lambda_k = lambdaList[k-1]
        sum_over_exp = sum_over_exp + lambda_k
        #print('lambda_k = ', lambda_k)
        k += 1
    P = 1 - math.exp((-1)*sum_over_exp*(v-u))
            
    return P

In [235]:
##Finding P_Miss_k   [Eq. E]
#Probability of a miss in certain segment (k)


In [236]:
##Verified
def P_miss_k(k):
    if k ==1:
        return 0
    
    #check for the maximum number of segments here
    if k > maxSegments:
        return 'Max number of segments is ', maxSegments
        
    product = 1
    l = 2
    while l <= k-1:
        #print(P_Miss_ml_uv(1,l-1,freshPeriodList[l-1-1], freshPeriodList[l-1]))
        #print(1,l-1,freshPeriodList[l-1-1],freshPeriodList[l-1])
        product = product*(1-P_Miss_ml_uv(1,l-1,freshPeriodList[l-1-1], freshPeriodList[l-1]))           
        l += 1
           
    tk = 0
    if k > len(freshPeriodList):
        tk = RP
    elif freshPeriodList[k-1] > RP:
        tk = RP
    else:
        tk = freshPeriodList[k-1]
        
    Pmiss_k = product * P_Miss_ml_uv(1,k-1,freshPeriodList[k-1-1], tk)
    
    return Pmiss_k

In [237]:
##E_Msk - is the realised number of misses of SLA s in segment k
#Verified

def E_Msk (s, k):
    result = 0.0
    if k <= s:
        result = 0
    else :
        result = P_miss_k(k) * P_Miss_sk_with_Miss_k(s,k)
    return result

In [238]:
##Ems is the realised number of misses of SLA s in a refresh period
def E_Ms(s):
    result = 0
    for k in range(1,maxSegments+1):
        result += E_Msk(s,k)    
    
    return result


In [239]:
#E[Ns]  - the expectation of number of requests of SLAs
#E[Nsk]is the realised number of requests of SLA s in segment k


In [240]:
def E_Nsk(s,k):
    
    E = 0
    
    if k == 1:
        E = lambdaList[s-1] * freshPeriodList[0]
    elif k >=2 and k <= s:
        #print("k>=2 and k <= s")
        product = 1
        for q in range(1,k-1+1):
            
            #print("q: ", q)
                       
            prob = 0
            if (q-1 < 1):  
                prob = 0
                #print("prob = 0")
            else:
                #print("u,v:", freshPeriodList[q-1-1],freshPeriodList[q-1])
                prob = P_Miss_ml_uv(1,q-1,freshPeriodList[q-1-1],freshPeriodList[q-1])
                #print("prob: ", prob)
            
            product = product * (1-prob)
         
        #print("product final: ", product)          
                
        product2 = 1
        for l in range(1,k+1): 
            
            #print("L: ", l)
           
            
            prob = 0
            if (l-1 < 1):  
                prob = 0
                #print("prod 2 prob = 0")
            else:
                #print("u,v:", freshPeriodList[l-1-1],freshPeriodList[l-1])
                prob = P_Miss_ml_uv(1,l-1,freshPeriodList[l-1-1],freshPeriodList[l-1])
                #print("prod 2 u,v:", freshPeriodList[l-1-1],freshPeriodList[l-1])
                #print("prob2: ", prob)
                
                      
            product2 = product2 * (1-prob)
            #print("product2: ", product2)
        
        #print("product2:", product2)
        
        ##substitution
        la_ = 0
        for q in range(1, k-1+1):
            la_ += lambdaList[q-1]
                
        #print("la_: ", la_)
        
        #lambdaList[k-1]
        right_integral_with_subst = \
        (lambdaList[s-1]/la_)*(1-(la_*(freshPeriodList[k-1]-freshPeriodList[k-1-1])+1)*\
        math.exp(-la_*(freshPeriodList[k-1]-freshPeriodList[k-1-1])))
        
                    
        #print("product: ",product)
        #print("right_integral_with_subst: ", right_integral_with_subst)
        
        #print("ENs: ", lambdaList[s-1]*(freshPeriodList[k-1] - freshPeriodList[k-1-1]))
        
        integral = product * (right_integral_with_subst) 
        #print("integral: ", integral)
        E = integral +product2*lambdaList[s-1]*(freshPeriodList[k-1] - freshPeriodList[k-1-1])
        #print("t_k-1; t_k: ",freshPeriodList[k-1-1],freshPeriodList[k-1])
        
    elif k > s:
        E = E_Msk(s,k)
        
    
    return E

In [241]:
def E_Ns(s):
    
    su = 0
    for k in range(1,maxSegments+1):
        su = su + E_Nsk(s,k)
    return su

In [242]:
ExpectedDuration = E_Ns(1)/lambdaList[0]
print("Expected duration of a cycle: ", ExpectedDuration)

Expected duration of a cycle:  1.13302352744846


In [243]:
NumberCycles = planningPeriod/ExpectedDuration
print("Number of cycles: ", NumberCycles)

Number of cycles:  529.5565232887835


In [244]:
def profitRP():
    result = 0
    for i in range(1,len(freshPeriodList)+1):
        result += (E_Ns(i)*requestPriceList[i-1] - E_Ms(i)*penaltyList[i-1])
    result = result - retrievalPrice
    return result


In [245]:
print(profitRP()*NumberCycles)

13238883.260753341


In [246]:
gapTime = 0.15

In [247]:
print(profitRP()*NumberCycles)

13238883.260753341


In [253]:
def profitCompute(g):
    global gapTime 
    global maxSegments
    global RP
    global freshPeriodList
    global freshPeriodListOrig
    maxSegments = 0
    gapTime = g
    print("gapTime: ", gapTime)
    RP = freshPeriodList[0] + gapTime
    maxSegments = countMaxSegments()
    print("maxSegments", maxSegments)
    freshPeriodList = freshPeriodListOrig.copy()
    periodsCut()
    print(freshPeriodList)
    ExpectedDuration = E_Ns(1)/lambdaList[0]
    NumberCycles = planningPeriod/ExpectedDuration
    result = profitRP()*NumberCycles
    print("result: ",result)
    return result

In [254]:
g = 0.0
resultList = []
while g <= 11.0:    
    res = profitCompute(g)
    s = (g,res)
    resultList.append(s)
    g += 0.05

gapTime:  0.0
RP:  1.148119540723993
1.148119540723993
maxSegments: 1
maxSegments 1
[1.148119540723993, 1.148119540723993, 1.148119540723993, 1.148119540723993, 1.148119540723993, 1.148119540723993, 1.148119540723993, 1.148119540723993, 1.148119540723993, 1.148119540723993]
result:  177742.8428033335
gapTime:  0.05
RP:  1.198119540723993
maxSegments: 11
maxSegments 11
[1.148119540723993, 1.198119540723993, 1.198119540723993, 1.198119540723993, 1.198119540723993, 1.198119540723993, 1.198119540723993, 1.198119540723993, 1.198119540723993, 1.198119540723993]
result:  179331.03676246761
gapTime:  0.1
RP:  1.2481195407239931
maxSegments: 11
maxSegments 11
[1.148119540723993, 1.2353714971882412, 1.2481195407239931, 1.2481195407239931, 1.2481195407239931, 1.2481195407239931, 1.2481195407239931, 1.2481195407239931, 1.2481195407239931, 1.2481195407239931]
result:  180751.39821170934
gapTime:  0.15000000000000002
RP:  1.298119540723993
maxSegments: 11
maxSegments 11
[1.148119540723993, 1.2353714

maxSegments 11
[1.148119540723993, 1.2353714971882412, 2.415473087993526, 2.5124435788965718, 2.722023056477838, 3.7868621337274377, 4.9481195407239875, 4.9481195407239875, 4.9481195407239875, 4.9481195407239875]
result:  191735.7598535917
gapTime:  3.8499999999999943
RP:  4.998119540723987
maxSegments: 11
maxSegments 11
[1.148119540723993, 1.2353714971882412, 2.415473087993526, 2.5124435788965718, 2.722023056477838, 3.7868621337274377, 4.998119540723987, 4.998119540723987, 4.998119540723987, 4.998119540723987]
result:  191731.99406972938
gapTime:  3.899999999999994
RP:  5.048119540723987
maxSegments: 11
maxSegments 11
[1.148119540723993, 1.2353714971882412, 2.415473087993526, 2.5124435788965718, 2.722023056477838, 3.7868621337274377, 5.048119540723987, 5.048119540723987, 5.048119540723987, 5.048119540723987]
result:  191728.6525370433
gapTime:  3.949999999999994
RP:  5.098119540723987
maxSegments: 11
maxSegments 11
[1.148119540723993, 1.2353714971882412, 2.415473087993526, 2.512443578

[1.148119540723993, 1.2353714971882412, 2.415473087993526, 2.5124435788965718, 2.722023056477838, 3.7868621337274377, 5.393024357881993, 6.135265515050152, 6.159169396795528, 9.243347449403675]
result:  191698.06432184734
gapTime:  8.14999999999998
RP:  9.298119540723974
maxSegments: 11
maxSegments 11
[1.148119540723993, 1.2353714971882412, 2.415473087993526, 2.5124435788965718, 2.722023056477838, 3.7868621337274377, 5.393024357881993, 6.135265515050152, 6.159169396795528, 9.243347449403675]
result:  191698.06432152484
gapTime:  8.199999999999982
RP:  9.348119540723975
maxSegments: 11
maxSegments 11
[1.148119540723993, 1.2353714971882412, 2.415473087993526, 2.5124435788965718, 2.722023056477838, 3.7868621337274377, 5.393024357881993, 6.135265515050152, 6.159169396795528, 9.243347449403675]
result:  191698.06432128066
gapTime:  8.249999999999982
RP:  9.398119540723975
maxSegments: 11
maxSegments 11
[1.148119540723993, 1.2353714971882412, 2.415473087993526, 2.5124435788965718, 2.72202305

In [255]:
print(resultList)

[(0.0, 177742.8428033335), (0.05, 179331.03676246761), (0.1, 180751.39821170934), (0.15000000000000002, 181960.86839626575), (0.2, 183063.11278635313), (0.25, 184071.62259990408), (0.3, 184997.71723852103), (0.35, 185850.96441804725), (0.39999999999999997, 186639.50640433337), (0.44999999999999996, 187370.3159569603), (0.49999999999999994, 188049.39903801944), (0.5499999999999999, 188681.95677115535), (0.6, 189272.5158980942), (0.65, 189825.034656965), (0.7000000000000001, 190342.98932027316), (0.7500000000000001, 190829.44539237552), (0.8000000000000002, 191287.11654803588), (0.8500000000000002, 191718.413705922), (0.9000000000000002, 192125.48611116098), (0.9500000000000003, 192510.2559049253), (1.0000000000000002, 192874.4473546483), (1.0500000000000003, 193219.6116828534), (1.1000000000000003, 193547.14824886658), (1.1500000000000004, 193858.3226934918), (1.2000000000000004, 194154.2825428155), (1.2500000000000004, 194436.0706767825), (1.3000000000000005, 194617.416943951), (1.3500